Calculate the adjacency matrix, the stochastic adjacency matrix, and the graph Laplacian for the network in `5_1_data.txt`

In [19]:
import numpy as np
import networkx as nx

G: nx.Graph = nx.read_adjlist('5_1_data.txt')
adj_matrix = nx.adjacency_matrix(G)
stochastic_matrix = adj_matrix / adj_matrix.sum(axis=1)
laplacian_matrix = nx.laplacian_matrix(G)
print(adj_matrix.todense())
print(stochastic_matrix.todense())
print(laplacian_matrix.todense())

[[0 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0.         0.04347826 0.02631579 ... 0.         0.         0.        ]
 [0.03333333 0.         0.02631579 ... 0.         0.         0.        ]
 [0.03333333 0.04347826 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[30 -1 -1 ...  0  0  0]
 [-1 23 -1 ...  0  0  0]
 [-1 -1 38 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0  1]]


Given the bipartite network in http://www.networkatlas.eu/exercises/5/2/data.txt, calculate the stochastic adjacency matrix
of its projection. Project along the axis of size 248. (Note: don’t
ignore the weights)

In [48]:
B = nx.read_weighted_edgelist('5_2_data.txt')
sets = nx.bipartite.basic.sets(B)

adjmat = nx.algorithms.bipartite.matrix.biadjacency_matrix(B, sets[0])

if adjmat.shape[0] == 248:
    adjmat_proj = adjmat.dot(adjmat.T)
else:
    adjmat_proj = adjmat.T.dot(adjmat)

adjmat_proj_stoc = (adjmat_proj / adjmat_proj.sum(axis=1)).toarray()
print(adjmat_proj_stoc)

[[0.06522125 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01203287 0.         ... 0.         0.01179963 0.01161316]
 [0.         0.         0.43697447 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.22845134 0.         0.        ]
 [0.         0.0052598  0.         ... 0.         0.02511096 0.00508076]
 [0.         0.00438317 0.         ... 0.         0.00430195 0.04028314]]


Calculate the eigenvalues and the right and left eigenvectors of
the stochastic adjacency obtained in the previous question. Make
sure to sort the eigenvalues in descending order (and sort the
eigenvectors accordingly). Only take the real part of eigenvalues
and eigenvectors, ignoring the imaginary part.

In [66]:
eigenvalues, eigenvectors = np.linalg.eig(adjmat_proj_stoc)
idx = eigenvalues.argsort()[::-1]
eigenvalues = np.real(eigenvalues[idx])
eigenvectors = np.real(eigenvectors[:,idx])

What is the average reciprocity in the network used in the previous question (https://www.networkatlas.eu/exercises/7/2/data.txt)? How many nodes have a reciprocity of zero?

In [73]:
G = nx.read_adjlist('7_2_data.txt', create_using=nx.DiGraph)
rec = nx.reciprocity(G)
zero_nodes = len([i for i in nx.reciprocity(G, G.nodes).values() if i == 0])

How many weakly and strongly connected component does the
network used in the previous question have? Compare their sizes,
in number of nodes, with the entire network. Which nodes are in
these two components?

In [91]:
strong = list(nx.strongly_connected_components(G))
weak = list(nx.weakly_connected_components(G))
strong_count = len(strong)
weak_count = len(weak)
print(f"Weak connected: {weak_count}")
print(f"Strong connected: {strong_count}")
strong_biggest = max(strong, key=lambda x: len(x))
weak_biggest = max(weak, key=lambda x: len(x))
print(f"Percentage of biggest strong: {len(strong_biggest)/len(G.nodes) * 100}")
print(f"Percentage of biggest weak: {len(weak_biggest)/len(G.nodes) * 100}")
print(f"Nodes in biggest strong: {strong_biggest}")
print(f"Nodes in biggest weak: {weak_biggest}")

Weak connected: 11
Strong connected: 52
Percentage of biggest strong: 97.58421231711466
Percentage of biggest weak: 98.84314392650562
Nodes in biggest strong: {'2625', '1447', '379', '2791', '2002', '464', '640', '405', '2677', '67', '2033', '623', '1353', '2074', '2332', '707', '2565', '2578', '1435', '2690', '967', '2179', '957', '2143', '2193', '320', '1947', '2562', '879', '2244', '2462', '1949', '1053', '2409', '657', '2369', '872', '630', '411', '792', '1856', '741', '1056', '112', '1841', '830', '689', '1560', '1112', '2280', '2820', '38', '925', '2726', '2281', '2782', '1200', '2403', '2761', '816', '329', '1933', '1686', '1345', '677', '1546', '1861', '2893', '2519', '2330', '1705', '1137', '56', '2883', '554', '2581', '1519', '1174', '1992', '549', '812', '2282', '553', '301', '2662', '2045', '1219', '135', '562', '2319', '1889', '2678', '1563', '153', '331', '2000', '1458', '2491', '2520', '1168', '1', '2449', '1912', '2621', '2475', '1351', '2537', '2171', '2064', '2768', '

Calculate the stationary distribution of the network at http://www.networkatlas.eu/exercises/8/1/data.txt in three ways: by
raising the stochastic adjacency to a high power, by looking at the
leading left eigenvector, and by normalizing the degree. Verify that
they are all equivalent.

In [108]:
from scipy.stats import pearsonr
G = nx.read_adjlist('8_1_data.txt')
# 1
adj_matrix = nx.adjacency_matrix(G).toarray()
stochastic_matrix = adj_matrix / adj_matrix.sum(axis=1)
stocpow = np.power(stochastic_matrix, 1000)
stationary2 = stocpow[0]

# 2
eigenvalues, eigenvectors = np.linalg.eig(stochastic_matrix)
idx = eigenvalues.argsort()[::]
eigenvalues = np.real(eigenvalues[idx])
eigenvectors = np.real(eigenvectors[idx])
stationary1 = np.real(eigenvectors[0])
stationary1 /= stationary1.sum()

# 3
stationary3 = np.matrix([G.degree(n) / (2 * len(G.edges)) for n in G.nodes])
print(pearsonr(np.array(stationary1)[0], np.array(stationary2).T[0]))
print(pearsonr(np.array(stationary2).T[:,0], np.array(stationary3).T[:,0]))
print(pearsonr(np.array(stationary1)[:,0], np.array(stationary3).T[:,0]))


TypeError: object of type 'numpy.float64' has no len()